<b> LOADING THE DATA <b>

In [0]:
dataset_year = 2020

In [0]:
# DO NOT CHANGE THE CONTENT OF THIS CELL
import os
import sys

fileroot = "clinicaltrial_" + str(dataset_year) + "_csv"
renamed_fileroot = "clinicaltrial_" + str(dataset_year) + ".csv"
mesh_csv = "/FileStore/tables/mesh.csv"
pharma_csv = "/FileStore/tables/pharma.csv"

# if 'dbruntime.dbutils' in sys.modules.keys():
try:
    dbutils.fs.ls("/FileStore/tables/" + renamed_fileroot)
except:
    dbutils.fs.cp("/FileStore/tables/" + fileroot + ".gz", "file:/tmp/")
    os.environ['fileroot'] = fileroot

In [0]:
%sh
gunzip /tmp/ /tmp/$fileroot.gz

gzip: /tmp/ is a directory -- ignored
gzip: /tmp/.gz.gz: No such file or directory


In [0]:
try:
    dbutils.fs.ls("file:/tmp/" + fileroot)
    dbutils.fs.mv("file:/tmp/" + fileroot, "/FileStore/tables/" + renamed_fileroot, True)
except:
    pass

In [0]:
dbutils.fs.ls("/FileStore/tables/")

Out[7]: [FileInfo(path='dbfs:/FileStore/tables/accounts.zip', name='accounts.zip', size=5297592),
 FileInfo(path='dbfs:/FileStore/tables/clinicaltrial_2019.csv', name='clinicaltrial_2019.csv', size=42400056),
 FileInfo(path='dbfs:/FileStore/tables/clinicaltrial_2019_csv.gz', name='clinicaltrial_2019_csv.gz', size=10060669),
 FileInfo(path='dbfs:/FileStore/tables/clinicaltrial_2020.csv', name='clinicaltrial_2020.csv', size=46318151),
 FileInfo(path='dbfs:/FileStore/tables/clinicaltrial_2020_csv.gz', name='clinicaltrial_2020_csv.gz', size=10981608),
 FileInfo(path='dbfs:/FileStore/tables/clinicaltrial_2021_csv.gz', name='clinicaltrial_2021_csv.gz', size=11921810),
 FileInfo(path='dbfs:/FileStore/tables/devicestatus.zip', name='devicestatus.zip', size=23873574),
 FileInfo(path='dbfs:/FileStore/tables/json_activations.zip', name='json_activations.zip', size=8411369),
 FileInfo(path='dbfs:/FileStore/tables/letter_frequencies.txt', name='letter_frequencies.txt', size=2593894),
 FileInfo(path

In [0]:
clinical_csv = "/FileStore/tables/" + renamed_fileroot
mesh_csv = "/FileStore/tables/mesh.csv"
pharma_csv = "/FileStore/tables/pharma.csv"

<b>PREPARE THE DATA<b>

USING DATAFRAME

In [0]:
from pyspark.sql.functions import *

In [0]:
clinicalDF = spark.read.option("header","true").option("inferSchema", "true").option("delimiter", '|').csv(clinical_csv)
clinicalDF.show(10)

+-----------+--------------------+--------------------+--------+----------+--------------------+----------+--------------------+--------------------+
|         Id|             Sponsor|              Status|   Start|Completion|                Type|Submission|          Conditions|       Interventions|
+-----------+--------------------+--------------------+--------+----------+--------------------+----------+--------------------+--------------------+
|NCT02758028|The University of...|          Recruiting|Aug 2005|  Nov 2021|      Interventional|  Apr 2016|                null|                null|
|NCT02751957|     Duke University|           Completed|Jul 2016|  Jul 2020|      Interventional|  Apr 2016|Autistic Disorder...|                null|
|NCT02758483|Universidade Fede...|           Completed|Mar 2017|  Jan 2018|      Interventional|  Apr 2016|   Diabetes Mellitus|                null|
|NCT02759848|Istanbul Medeniye...|           Completed|Jan 2012|  Dec 2014|       Observational|  Ma

In [0]:
meshDF = spark.read.option("header","true").option("inferSchema", "true").csv(mesh_csv)
meshDF.show(10)

+----------+-------------------+
|      term|               tree|
+----------+-------------------+
|Calcimycin|D03.633.100.221.173|
|   A-23187|D03.633.100.221.173|
|   Temefos|D02.705.400.625.800|
|   Temefos|D02.705.539.345.800|
|   Temefos|D02.886.300.692.800|
|     Abate|D02.705.400.625.800|
|     Abate|D02.705.539.345.800|
|     Abate|D02.886.300.692.800|
|     Difos|D02.705.400.625.800|
|     Difos|D02.705.539.345.800|
+----------+-------------------+
only showing top 10 rows



In [0]:
pharmaDF = spark.read.option("header","true").option("inferSchema", "true").csv(pharma_csv)


<b>ANALYSE THE DATA<b>

In [0]:
# QUESTION1 - The number of studies in the dataset
clinicalDF.distinct().count()

Out[13]: 356466

In [0]:
# QUESTION2 - List all the Type of studies in the dataset along with the frequencies of each type
typesFromDF = clinicalDF.groupBy("Type").count().orderBy("count", ascending=False)
typesFromDF.show()

+--------------------+------+
|                Type| count|
+--------------------+------+
|      Interventional|277631|
|       Observational| 71434|
|Observational [Pa...|  7332|
|     Expanded Access|    69|
+--------------------+------+



In [0]:
# QUESTION3 - The top 5 Conditions with their frequencies
splitConditionsFromDF = split(regexp_replace(col("Conditions"), "(^\[)|(\]$)", ""), ",")
explodeConditionsFromDF = clinicalDF.withColumn(
    "Conditions", explode(splitConditionsFromDF)
)

topConditionsFromDF = explodeConditionsFromDF.groupBy("Conditions").count().orderBy("count", ascending=False)
topConditionsFromDF.show(5, truncate=False)

+-----------------+-----+
|Conditions       |count|
+-----------------+-----+
|Carcinoma        |12245|
|Diabetes Mellitus|10425|
|Neoplasms        |8534 |
|Breast Neoplasms |8009 |
|Syndrome         |7419 |
+-----------------+-----+
only showing top 5 rows



In [0]:
# QUESTION4 - The 5 most frequent roots from the hierarchy codes
rootsFromDF = meshDF.join(explodeConditionsFromDF, meshDF.term == explodeConditionsFromDF.Conditions).\
                    select("tree", "term", "Conditions").withColumn('tree', substring('tree', 1,3)).\
                    groupBy("tree").count().orderBy("count", ascending=False)
rootsFromDF.show(10)

+----+------+
|tree| count|
+----+------+
| C04|133091|
| C23|124589|
| C01| 94293|
| C14| 88065|
| C10| 83894|
| C06| 79382|
| C08| 61138|
| C13| 39316|
| C18| 38439|
| C12| 37294|
+----+------+
only showing top 10 rows



In [0]:
# QUESTION 5 - The 10 most common sponsors that are not pharmaceutical companies with the number of clinical trials they have sponsored
nonPharmaSponsorsDF = clinicalDF.join(pharmaDF, pharmaDF.Parent_Company == clinicalDF.Sponsor, "left").filter(pharmaDF.Parent_Company.isNull())
mostCommonNonPharmaSponsorsDF = nonPharmaSponsorsDF.filter(nonPharmaSponsorsDF.Status!="Active").groupBy("Sponsor").count().orderBy("count", ascending=False)

mostCommonNonPharmaSponsorsDF.show(10, truncate=False)

+---------------------------------------+-----+
|Sponsor                                |count|
+---------------------------------------+-----+
|National Cancer Institute (NCI)        |3100 |
|M.D. Anderson Cancer Center            |2238 |
|Merck Sharp & Dohme Corp.              |2184 |
|Mayo Clinic                            |2097 |
|Assistance Publique - Hôpitaux de Paris|2043 |
|Novartis Pharmaceuticals               |1962 |
|Massachusetts General Hospital         |1823 |
|Assiut University                      |1806 |
|Hoffmann-La Roche                      |1761 |
|National Taiwan University Hospital    |1720 |
+---------------------------------------+-----+
only showing top 10 rows



In [0]:
# QUESTION 6 - Number of completed studies each month in a given year
yearfilter = "%" + str(dataset_year)

completedStudiesDF = clinicalDF.filter(clinicalDF.Status=="Completed").filter(col("Completion").like(yearfilter)).\
                                    withColumn('Completion', substring('Completion', 1,3)).groupBy("Completion").count()
numberOfCompletedStudiesDF = completedStudiesDF.sort(unix_timestamp(col("Completion"),"MMM"))

numberOfCompletedStudiesDF.show()

+----------+-----+
|Completion|count|
+----------+-----+
|       Jan| 1544|
|       Feb| 1286|
|       Mar| 1740|
|       Apr| 1080|
|       May| 1176|
|       Jun| 1424|
|       Jul| 1237|
|       Aug| 1126|
|       Sep| 1167|
|       Oct| 1176|
|       Nov| 1078|
|       Dec| 2084|
+----------+-----+



In [0]:
# Values to Plot on Bar chart
months = numberOfCompletedStudiesDF.select("Completion").rdd.map(lambda row: row["Completion"]).collect()
counts = numberOfCompletedStudiesDF.select("count").rdd.map(lambda row: row["count"]).collect()

months

Out[19]: ['Jan',
 'Feb',
 'Mar',
 'Apr',
 'May',
 'Jun',
 'Jul',
 'Aug',
 'Sep',
 'Oct',
 'Nov',
 'Dec']

<b>VISUALISE RESULTS<b>

In [0]:
# Q2: Install bokeh
%pip install bokeh

Python interpreter will be restarted.
You should consider upgrading via the '/local_disk0/.ephemeral_nfs/envs/pythonEnv-625b2628-d05e-40c5-831e-a7e11953fede/bin/python -m pip install --upgrade pip' command.
Python interpreter will be restarted.


In [0]:
from bokeh.io import output_file, show
from bokeh.plotting import figure
from bokeh.embed import file_html
from bokeh.resources import CDN

months = numberOfCompletedStudiesDF.select("Completion").rdd.map(lambda row: row["Completion"]).collect()
counts = numberOfCompletedStudiesDF.select("count").rdd.map(lambda row: row["count"]).collect()

p = figure(x_range=months, height=250, title="Completed studies each month in a given year",
           toolbar_location=None, tools="")

p.vbar(x=months, top=counts, width=0.7)

p.xgrid.grid_line_color = None
p.y_range.start = 0

html = file_html(p, CDN, "plot")
displayHTML(html)

---------------------------------------------------------------------------
NameError                                 Traceback (most recent call last)
<command-4038768821771782> in <module>
      4 from bokeh.resources import CDN
      5 
----> 6 months = numberOfCompletedStudiesDF.select("Completion").rdd.map(lambda row: row["Completion"]).collect()
      7 counts = numberOfCompletedStudiesDF.select("count").rdd.map(lambda row: row["count"]).collect()
      8 

NameError: name 'numberOfCompletedStudiesDF' is not defined

USING RDD